### GENERATE TRAJECTORIES USING **DOUBLE-WELL POTENTIAL**

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy.constants import Boltzmann as kB
from numpy.random import randn as gauss
from numpy.random import rand as uniform

### Physical parameters 
R = 1e-7                                # Radius of the Brownian particle [m]
eta = 0.001                             # Viscosity of the medium [kg m^-1 s^-1]
T = 300                                 # Temperature [K]
L0 = 2e-6                               # Reference distance from middle to one minimum [m]
H0 = kB*300                             # Barrier height [Joule]
gamma0 = 6 * np.pi * eta * R            # Reference friction coefficient [kg s^-1]

### Simulation parameters
N = 1000                   # Number of samples of the trajectory
Dt = 1e-2                  # Timestep 
oversampling = 5           # Simulation oversampling
offset = 1000              # Number of equilibration timesteps
batch_size = 32            # Number of trajectories

### Define functions to scale and rescale inputs
scale_inputs = lambda x: x * 1e+6                    # Scales input trajectory to order 1
rescale_inputs = lambda scaled_x: scaled_x * 1e-6    # Rescales input trajectory to physical units

### Define function to scale and rescale targets
scale_targets = lambda L, H: [L/L0 -1, np.log(H / H0)]                        # Scales targets to order 1
rescale_targets = lambda scaled_L, scaled_H: [(1 + scaled_L)*L0*1e6, 
                                              np.exp(scaled_H) * H0/kB/300] # Inverse of targets_scaling

def simulate_trajectory(batch_size=batch_size, 
                        T=T,
                        H0=H0,
                        L0=L0,
                        gamma0=gamma0,
                        N=N, 
                        Dt=Dt, 
                        oversampling=oversampling, 
                        offset=offset):#, 
                        #scale_inputs=scale_inputs, 
                        #scale_targets=scale_targets):
    
    ### Randomize trajectory parameters
    L = L0 * (uniform(batch_size)+.5) 
    H = H0 * 10**(uniform(batch_size)*1.75 - .75)       # Generates random values for computing the stiffness
    gamma = gamma0 * (uniform(batch_size) * .1 + .95)   # Marginal randomization of friction coefficient to tolarate small changes

    ### Simulate
    dt = Dt / oversampling                 # time step of the simulation
    x = np.zeros((batch_size, N))          # initialization of the x array
    k0 = 4*H/L**2 
    k1 = 4*H/L**4
    D = kB * T / gamma                     # diffusion coefficient
    C1 = +k0 / gamma * dt
    C2 = -k1 / gamma * dt                  # drift coefficient of the Langevin equation
    C3 = np.sqrt(2 * D * dt)               # random walk coefficient of the Langevin equation
    X = x[:, 0]
    n = 0
    
    for t in range(offset):                      # Offset (for some prerun before running)
        X = X + C1 * X + C2 * X**3 + C3 * gauss(batch_size)
        #X = X - (2 * X**3) + (12 * X**2) - (18 * X) + 3 + (C3 * gauss(batch_size))
        
    for t in range(N * oversampling):            # Simulation                
        X = X + C1 * X + C2 * X**3 + C3 * gauss(batch_size)
        #X = X - (2 * X**3) + (12 * X**2) - (18 * X) + 3 + (C3 * gauss(batch_size))
        if t % oversampling == 0:                # We save every oversampling^th values 
            x[:, n] = X 
            n += 1

    inputs = scale_inputs(x)
    targets = np.swapaxes(scale_targets(*[L, H]),0,1)
    target_reals = np.swapaxes([L*1e6, H/kB/300],0,1)

    return inputs, targets, target_reals

##### TRAIN THE DEEP NEURAL NETWORK

In [2]:
def train_deep_learning_network(
    network,
    simulate_trajectory,
    sample_sizes = (32, 128, 512),#(32, 128, 512, 2048),
    iteration_numbers = (3001, 2001, 1001),#(1001, 2001, 3001),#(3001, 2001, 1001, 101),
    verbose=.1):
    """Train a deep learning network.
    
    Input:
    network: deep learning network
    simulate_trajectory: trajectory generator function
    sample_sizes: sizes of the batches of trajectories used in the training [tuple of positive integers]
    iteration_numbers: numbers of batches used in the training [tuple of positive integers]
    verbose: frequency of the update messages [number between 0 and 1]
        
    Output:
    training_history: dictionary with training history
    """  
    
    import numpy as np
    from time import time
     
    training_history = {}
    training_history['Sample Size'] = []
    training_history['Iteration Number'] = []
    training_history['Iteration Time'] = []
    training_history['MSE'] = []
    training_history['MAE'] = []
    
    for sample_size, iteration_number in zip(sample_sizes, iteration_numbers):
        for iteration in range(iteration_number):
            
            # measure initial time for iteration
            initial_time = time()

            # generate trajectories and targets
            network_blocksize = network.get_layer(index=0).get_config()['batch_input_shape'][1:][1]                        
            number_of_outputs = network.get_layer(index=-1).get_config()['units']
            output_shape = (sample_size, number_of_outputs)
            targets = np.zeros(output_shape)
            
            
            batch_size = sample_size
            trajectory, target, target_real = simulate_trajectory(batch_size)
            #trajectory = trajectory.scaled_values
            trajectory_dimensions = [sample_size, round(trajectory.size/network_blocksize/sample_size), network_blocksize]
            trajectories = np.array(trajectory).reshape(trajectory_dimensions)
            targets = target#.scaled_values
                
                

            # training
            history = network.fit(trajectories,
                                targets,
                                epochs=1, 
                                batch_size=sample_size,
                                verbose=False)
                        
            # measure elapsed time during iteration
            iteration_time = time() - initial_time

            # record training history
            mse = history.history['mse'][0]
            mae = history.history['mae'][0]
                        
            training_history['Sample Size'].append(sample_size)
            training_history['Iteration Number'].append(iteration)
            training_history['Iteration Time'].append(iteration_time)
            training_history['MSE'].append(mse)
            training_history['MAE'].append(mae)

            if not(iteration%int(verbose**-1)):
                print('Sample size %6d   iteration number %6d   MSE %10.4f   MAE %10.4f   Time %10f ms' % (sample_size, iteration + 1, mse, mae, iteration_time * 1000))
                
    return training_history

##### PLOT THE PERFORMANCE

In [3]:
def plot_learning_performance(training_history, number_of_timesteps_for_average = 100, figsize=(20,20)):
    """Plot the learning performance of the deep learning network.
    
    Input:
    training_history: dictionary with training history, typically obtained from train_deep_learning_network()
    number_of_timesteps_for_average: length of the average [positive integer number]
    figsize: figure size [list of two positive numbers]
        
    Output: none
    """    

    import matplotlib.pyplot as plt
    from numpy import convolve, ones
    
    plt.figure(figsize=figsize)

    plt.subplot(5, 1, 1)
    plt.semilogy(training_history['MSE'], 'k')
    plt.semilogy(convolve(training_history['MSE'], ones(number_of_timesteps_for_average) / number_of_timesteps_for_average, mode='valid'), 'r')
    plt.ylabel('MSE', fontsize=24)
    plt.xlabel('Epochs', fontsize=24)

    plt.subplot(5, 1, 2)
    plt.semilogy(training_history['MAE'], 'k')
    plt.semilogy(convolve(training_history['MAE'], ones(number_of_timesteps_for_average) / number_of_timesteps_for_average, mode='valid'), 'r')
    plt.ylabel('MAE', fontsize=24)
    plt.xlabel('Epochs', fontsize=24)
    plt.show()

##### TEST THE DNN ON NEW SIMULATED TRAJECTORIES

In [4]:
def predict(network, trajectory):
    """ Predict parameters of the force field from the trajectory using the deep learnign network.
    
    Inputs:
    network: deep learning network
    image: trajectroy [numpy array of real numbers]
    
    Output:
    predicted_targets: predicted parameters of the calibrated force field [1D numpy array containing outputs]
    """
    
    from numpy import reshape
    
    network_blocksize = network.get_layer(index=0).get_config()['batch_input_shape'][1:][1]
    predicted_targets = network.predict(reshape(trajectory, [1,round(trajectory.size/network_blocksize),network_blocksize]))   
        
    return predicted_targets


def test_performance(simulate_trajectory, network, rescale_targets, number_of_predictions_to_show=100):#, dt = 1e-1):

    
    network_blocksize = network.get_layer(index=0).get_config()['batch_input_shape'][1:][1]


    predictions_scaled = []
    predictions_physical = []

    batch_size = number_of_predictions_to_show
    trajectory, targets, targets_real = simulate_trajectory(batch_size)
    targets_physical = list(targets_real)#targets.values)
    targets_scaled = list(targets)#.scaled_values)
    #trajectory = trajectory.scaled_values
    trajectory_dimensions = [number_of_predictions_to_show, round(trajectory.size/network_blocksize/number_of_predictions_to_show) , network_blocksize]
    trajectories = np.array(trajectory).reshape(trajectory_dimensions)
       

    for i in range(number_of_predictions_to_show):
        predictions = predict(network, trajectories[i])


        predictions_scaled.append(predictions[0])
        predictions_physical.append(rescale_targets(*predictions[0]))

    number_of_outputs = network.get_layer(index=-1).get_config()['units']    

    targets_physical = np.array(targets_physical).transpose()
    targets_scaled = np.array(targets_scaled).transpose()
    predictions_scaled = np.array(predictions_scaled).transpose()
    predictions_physical = np.array(predictions_physical).transpose()

    # Do not show results at the edges of the training range 

    if number_of_outputs>1:

        ind = np.isfinite(targets_scaled[0])
        for target_number in range(number_of_outputs):
            target_max = .9 * np.max(targets_scaled[target_number]) + .1 * np.min(targets_scaled[target_number])
            target_min = .1 * np.max(targets_scaled[target_number]) + .9 * np.min(targets_scaled[target_number])
            ind = np.logical_and(ind, targets_scaled[target_number] < target_max)
            ind = np.logical_and(ind, targets_scaled[target_number] > target_min)
    else:
        target_max = .9 * np.max(targets_scaled) + .1 * np.min(targets_scaled)
        target_min = .1 * np.max(targets_scaled) + .9 * np.min(targets_scaled)
        ind = np.logical_and(targets_scaled < target_max, targets_scaled > target_min)

    return targets_scaled, targets_physical, predictions_scaled, predictions_physical


def plot_test_performance(targets_scaled, targets_physical, predictions_scaled, predictions_physical, network):
    
    import matplotlib.pyplot as plt
    import numpy as np
    #from . import predict

    number_of_outputs = network.get_layer(index=-1).get_config()['units']    
    
    if number_of_outputs>1:

        for target_number in range(number_of_outputs):
            plt.figure(figsize=(20, 10))

            plt.subplot(121)
            plt.plot(targets_scaled[target_number],
                     predictions_scaled[target_number],
                     '.')
            #plt.xlabel(targets.scalings[target_number], fontsize=18)
            #plt.ylabel('Predicted ' + targets.scalings[target_number], fontsize=18)
            plt.axis('square')
            plt.title('Prediction performance in scaled units', fontsize=18)

            plt.subplot(122)
            plt.plot(targets_physical[target_number],
                     predictions_physical[target_number],
                    '.')
            #plt.xlabel(targets.names[target_number], fontsize=18)
            #plt.ylabel('Predicted ' + targets.names[target_number], fontsize=18)
            plt.axis('square')
            plt.title('Prediction performance in real units', fontsize=18)


    else: 
        plt.figure(figsize=(20, 10))

        plt.subplot(121)
        plt.plot(targets_scaled[ind],
                 predictions_scaled.transpose()[ind],
                 '.')
        #plt.xlabel(targets.scalings[0], fontsize=18)
        #plt.ylabel('Predicted ' + targets.scalings[0], fontsize=18)
        plt.axis('square')
        plt.title('Prediction performance in scaled units', fontsize=18)

        plt.subplot(122)
        plt.plot(targets_physical[ind],
                 predictions_physical.transpose()[ind],
                '.')
        #plt.xlabel(targets.names[0], fontsize=18)
        #plt.ylabel('Predicted ' + targets.names[0], fontsize=18)
        plt.axis('square')
        plt.title('Prediction performance in real units', fontsize=18)

## LSTM

CREATE MODEL

In [5]:
def create_deep_learning_network_LSTM(
    input_shape=(None, 50),
    lstm_layers_dimensions=(100, 50),
    number_of_outputs=2) :
    """Creates and compiles a deep learning network.
    
    Inputs:    
    input_shape: Should be the same size of the input trajectory []
    lstm_layers_dimensions: number of neurons in each LSTM layer [tuple of positive integers]
        
    Output:
    network: deep learning network
    """    

    from keras import models, layers, optimizers
    
    ### INITIALIZE DEEP LEARNING NETWORK
    network = models.Sequential()

    ### CONVOLUTIONAL BASIS
    for lstm_layer_number, lstm_layer_dimension in zip(range(len(lstm_layers_dimensions)), lstm_layers_dimensions):

        # add LSTM layer
        lstm_layer_name = 'lstm_' + str(lstm_layer_number + 1)
        if lstm_layer_number + 1 < len(lstm_layers_dimensions): # All layers but last
            lstm_layer = layers.LSTM(lstm_layer_dimension,
                                     return_sequences=True,
                                     dropout=0,
                                     recurrent_dropout=0,
                                     input_shape=input_shape,
                                     name=lstm_layer_name)
        else: # Last layer
            lstm_layer = layers.LSTM(lstm_layer_dimension,
                                     return_sequences=False,
                                     dropout=0,
                                     recurrent_dropout=0,
                                     input_shape=input_shape,
                                     name=lstm_layer_name)

        network.add(lstm_layer)
    # OUTPUT LAYER
    output_layer = layers.Dense(number_of_outputs, name='output')
    network.add(output_layer)
    
    network.compile(optimizer=optimizers.Adam(lr=1e-3), loss='mse', metrics=['mse', 'mae'])
    
    return network

In [6]:
### Create deep learning network
networkLSTM = create_deep_learning_network_LSTM()

### Print deep learning network summary
networkLSTM.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, None, 100)         60400     
                                                                 
 lstm_2 (LSTM)               (None, 50)                30200     
                                                                 
 output (Dense)              (None, 2)                 102       
                                                                 
Total params: 90,702
Trainable params: 90,702
Non-trainable params: 0
_________________________________________________________________


C:\Users\fap_9\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


#### TRAIN MODEL

In [7]:
%%time

training_history_LSTM = train_deep_learning_network(networkLSTM, simulate_trajectory)

Sample size     32   iteration number      1   MSE     1.1166   MAE     0.8116   Time 3794.589281 ms
Sample size     32   iteration number     11   MSE     0.5990   MAE     0.5656   Time 153.102160 ms
Sample size     32   iteration number     21   MSE     0.4586   MAE     0.4816   Time 134.467840 ms
Sample size     32   iteration number     31   MSE     0.3927   MAE     0.4522   Time 145.557404 ms
Sample size     32   iteration number     41   MSE     0.4400   MAE     0.4437   Time 281.499147 ms
Sample size     32   iteration number     51   MSE     0.2444   MAE     0.3128   Time 188.031673 ms
Sample size     32   iteration number     61   MSE     0.3459   MAE     0.4352   Time 191.952229 ms
Sample size     32   iteration number     71   MSE     0.3998   MAE     0.4177   Time 126.492977 ms
Sample size     32   iteration number     81   MSE     0.4186   MAE     0.4408   Time 155.524492 ms
Sample size     32   iteration number     91   MSE     0.5929   MAE     0.5073   Time 147.021532 ms

Sample size     32   iteration number    821   MSE     0.2045   MAE     0.2919   Time 175.335646 ms
Sample size     32   iteration number    831   MSE     0.1595   MAE     0.2604   Time 132.042408 ms
Sample size     32   iteration number    841   MSE     0.1764   MAE     0.2932   Time 145.766258 ms
Sample size     32   iteration number    851   MSE     0.1468   MAE     0.2580   Time 234.969616 ms
Sample size     32   iteration number    861   MSE     0.1071   MAE     0.2293   Time 151.778460 ms
Sample size     32   iteration number    871   MSE     0.2332   MAE     0.3026   Time 140.320063 ms
Sample size     32   iteration number    881   MSE     0.1344   MAE     0.2561   Time 280.319691 ms
Sample size     32   iteration number    891   MSE     0.0948   MAE     0.2101   Time 173.815489 ms
Sample size     32   iteration number    901   MSE     0.1487   MAE     0.2531   Time 193.648815 ms
Sample size     32   iteration number    911   MSE     0.2486   MAE     0.3286   Time 142.672062 ms


Sample size     32   iteration number   1641   MSE     0.0757   MAE     0.1875   Time 159.538746 ms
Sample size     32   iteration number   1651   MSE     0.1657   MAE     0.2718   Time 171.832323 ms
Sample size     32   iteration number   1661   MSE     0.1359   MAE     0.2348   Time 220.942974 ms
Sample size     32   iteration number   1671   MSE     0.1449   MAE     0.2663   Time 192.127466 ms
Sample size     32   iteration number   1681   MSE     0.1356   MAE     0.2519   Time 336.708069 ms
Sample size     32   iteration number   1691   MSE     0.1541   MAE     0.2460   Time 247.748852 ms
Sample size     32   iteration number   1701   MSE     0.1275   MAE     0.2189   Time 334.475040 ms
Sample size     32   iteration number   1711   MSE     0.1074   MAE     0.2178   Time 202.265024 ms
Sample size     32   iteration number   1721   MSE     0.1094   MAE     0.2145   Time 301.887751 ms
Sample size     32   iteration number   1731   MSE     0.1367   MAE     0.2457   Time 197.019577 ms


Sample size     32   iteration number   2461   MSE     0.0972   MAE     0.1905   Time 440.001011 ms
Sample size     32   iteration number   2471   MSE     0.1165   MAE     0.2241   Time 221.257210 ms
Sample size     32   iteration number   2481   MSE     0.1228   MAE     0.2360   Time 219.857693 ms
Sample size     32   iteration number   2491   MSE     0.1051   MAE     0.1990   Time 219.539881 ms
Sample size     32   iteration number   2501   MSE     0.1500   MAE     0.2270   Time 204.120636 ms
Sample size     32   iteration number   2511   MSE     0.0961   MAE     0.1995   Time 227.429152 ms
Sample size     32   iteration number   2521   MSE     0.1346   MAE     0.2447   Time 312.337399 ms
Sample size     32   iteration number   2531   MSE     0.1224   MAE     0.2421   Time 216.217279 ms
Sample size     32   iteration number   2541   MSE     0.1132   MAE     0.2002   Time 251.365185 ms
Sample size     32   iteration number   2551   MSE     0.1288   MAE     0.2443   Time 267.016411 ms


Sample size    128   iteration number    271   MSE     0.1076   MAE     0.2012   Time 541.705608 ms
Sample size    128   iteration number    281   MSE     0.1178   MAE     0.2213   Time 862.441540 ms
Sample size    128   iteration number    291   MSE     0.1235   MAE     0.2316   Time 563.331366 ms
Sample size    128   iteration number    301   MSE     0.1220   MAE     0.2239   Time 466.686010 ms
Sample size    128   iteration number    311   MSE     0.0999   MAE     0.1987   Time 594.737053 ms
Sample size    128   iteration number    321   MSE     0.1072   MAE     0.2110   Time 672.722578 ms
Sample size    128   iteration number    331   MSE     0.0935   MAE     0.1947   Time 696.800947 ms
Sample size    128   iteration number    341   MSE     0.1059   MAE     0.2135   Time 610.518456 ms
Sample size    128   iteration number    351   MSE     0.0794   MAE     0.1804   Time 675.637722 ms
Sample size    128   iteration number    361   MSE     0.1144   MAE     0.2147   Time 667.191505 ms


Sample size    128   iteration number   1091   MSE     0.1066   MAE     0.2091   Time 608.335495 ms
Sample size    128   iteration number   1101   MSE     0.1009   MAE     0.1970   Time 612.133980 ms
Sample size    128   iteration number   1111   MSE     0.1002   MAE     0.1934   Time 559.406757 ms
Sample size    128   iteration number   1121   MSE     0.1192   MAE     0.2200   Time 577.830076 ms
Sample size    128   iteration number   1131   MSE     0.1070   MAE     0.2034   Time 590.687752 ms
Sample size    128   iteration number   1141   MSE     0.0960   MAE     0.1953   Time 592.143536 ms
Sample size    128   iteration number   1151   MSE     0.1090   MAE     0.2092   Time 557.466269 ms
Sample size    128   iteration number   1161   MSE     0.1157   MAE     0.2089   Time 587.612629 ms
Sample size    128   iteration number   1171   MSE     0.1075   MAE     0.2069   Time 607.295275 ms
Sample size    128   iteration number   1181   MSE     0.1304   MAE     0.2149   Time 603.923798 ms


Sample size    128   iteration number   1911   MSE     0.1393   MAE     0.2334   Time 783.941507 ms
Sample size    128   iteration number   1921   MSE     0.1341   MAE     0.2243   Time 796.834469 ms
Sample size    128   iteration number   1931   MSE     0.1214   MAE     0.2113   Time 720.352173 ms
Sample size    128   iteration number   1941   MSE     0.0870   MAE     0.1869   Time 698.848486 ms
Sample size    128   iteration number   1951   MSE     0.1048   MAE     0.2065   Time 786.649704 ms
Sample size    128   iteration number   1961   MSE     0.1259   MAE     0.2290   Time 780.730247 ms
Sample size    128   iteration number   1971   MSE     0.1001   MAE     0.1954   Time 764.817715 ms
Sample size    128   iteration number   1981   MSE     0.1004   MAE     0.1927   Time 710.824728 ms
Sample size    128   iteration number   1991   MSE     0.0921   MAE     0.1921   Time 842.901468 ms
Sample size    128   iteration number   2001   MSE     0.1174   MAE     0.2295   Time 1070.111513 ms

Sample size    512   iteration number    721   MSE     0.0929   MAE     0.1813   Time 2394.792795 ms
Sample size    512   iteration number    731   MSE     0.1005   MAE     0.1909   Time 1858.644247 ms
Sample size    512   iteration number    741   MSE     0.1061   MAE     0.1987   Time 2397.799253 ms
Sample size    512   iteration number    751   MSE     0.1030   MAE     0.1989   Time 1963.108301 ms
Sample size    512   iteration number    761   MSE     0.0997   MAE     0.1995   Time 2122.920513 ms
Sample size    512   iteration number    771   MSE     0.1056   MAE     0.1990   Time 2553.689718 ms
Sample size    512   iteration number    781   MSE     0.0917   MAE     0.1872   Time 2175.392628 ms
Sample size    512   iteration number    791   MSE     0.1010   MAE     0.1936   Time 1824.226379 ms
Sample size    512   iteration number    801   MSE     0.0977   MAE     0.1892   Time 2470.207453 ms
Sample size    512   iteration number    811   MSE     0.1037   MAE     0.2047   Time 2440.

#### TEST MODEL

In [8]:
%%time

number_of_predictions_to_show = 1000
prediction_LSTM_test = test_performance(simulate_trajectory, networkLSTM, rescale_targets, number_of_predictions_to_show)

1/1 [==============================] - 0s 50ms/step


1/1 [==============================] - 0s 24ms/step


1/1 [==============================] - 0s 19ms/step


1/1 [==============================] - 0s 48ms/step


1/1 [==============================] - 0s 50ms/step


1/1 [==============================] - 0s 35ms/step


1/1 [==============================] - 0s 52ms/step
CPU times: total: 25.7 s
Wall time: 1min 23s


#### Save data

In [ ]:
# save train data



### Plot results

In [ ]:
### Plot learning performance
number_of_timesteps_for_average = 100
plot_learning_performance(training_history_LSTM, number_of_timesteps_for_average)

In [ ]:
### Plot test performance
plot_test_performance(prediction_LSTM_test[0], prediction_LSTM_test[1], prediction_LSTM_test[2], prediction_LSTM_test[3], networkLSTM)